# Token counting
LangChain offers a context manager that allows you to count tokens.

## OpenAI example

In [3]:
import asyncio

from langchain.callbacks import get_openai_callback
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
with get_openai_callback() as cb:
    llm("What is the square root of 4?")

total_tokens = cb.total_tokens
assert total_tokens > 0

with get_openai_callback() as cb:
    llm("What is the square root of 4?")
    llm("What is the square root of 4?")

assert cb.total_tokens == total_tokens * 2

# You can kick off concurrent runs from within the context manager
with get_openai_callback() as cb:
    await asyncio.gather(
        *[llm.agenerate(["What is the square root of 4?"]) for _ in range(3)]
    )

assert cb.total_tokens == total_tokens * 3

# The context manager is concurrency safe
task = asyncio.create_task(llm.agenerate(["What is the square root of 4?"]))
with get_openai_callback() as cb:
    await llm.agenerate(["What is the square root of 4?"])

await task
assert cb.total_tokens == total_tokens

## Bedrock example

In [4]:
from langchain.callbacks import get_bedrock_token_count_callback
from langchain.llms import Bedrock
import anthropic

In [5]:
llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    region_name="us-east-1",
    model_kwargs={"temperature": 0.0},
)

In [6]:
with get_bedrock_token_count_callback() as cb:
    text = "\n\nHuman: What is the square root of 4?\n\nAssistant: "
    answer = llm(text)
    print(answer)  # check if invocations return same response
    answer = llm(text)
    print(answer)  # check if invocations return same response

    total_tokens = cb.total_tokens
    print(cb.total_tokens, cb.prompt_tokens, cb.completion_tokens)
    client = anthropic.Anthropic()  # check count with Anthropic's
    assert cb.prompt_tokens == client.count_tokens(text) * 2

    # to check total token count we need to include
    # generation stop sequence that is not included in answers
    full_text = text + "The square root of 4 is 2." + "\n\nHuman: "
    total_token_count = client.count_tokens(full_text) * 2

 The square root of 4 is 2.
 The square root of 4 is 2.
58 34 24
